## Практическое задание к уроку № 5 по теме "Рекуррентные нейронные сети".

**Задание 1.**

*Измените параметры нейронной сети, работающей с датасетом imdb так,  
чтобы улучшить ее точность. Приложите анализ.*

Возьмём за основу код с урока:

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding
from keras.layers import GRU, LSTM
from keras.datasets import imdb
import numpy as np

In [3]:
MAX_FEATURES = 20000
MAXLEN = 256
BATCH_SIZE = 32
RANDOM_SEED = 29
EPOCHS = 2

tf.random.set_seed(RANDOM_SEED)

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=MAX_FEATURES)

x_train = sequence.pad_sequences(x_train, maxlen=MAXLEN)
x_test = sequence.pad_sequences(x_test, maxlen=MAXLEN)

model = Sequential()
model.add(Embedding(MAX_FEATURES, MAXLEN))
model.add(LSTM(16))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5, seed=RANDOM_SEED))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS) 

score = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=0)

print('Тестовая точность:', score[1])

Epoch 1/2
782/782 [==============================] - 50s 62ms/step - loss: 0.4313 - accuracy: 0.8078
Epoch 2/2
782/782 [==============================] - 49s 63ms/step - loss: 0.2285 - accuracy: 0.9206
Тестовая точность: 0.8735600113868713


<ins>Вывод:</ins>  
В сравнении с результатом, полученным на уроке, удалось  
повысить точность предсказания на 3%. Для этого были внесены  
следующие изменения:  
- увеличено максимальное количество слов в обзорах  
с 80 до 256 (maxlen);  
- снижен размер batch с 64 до 32;  
- увеличено количество эпох с 1 до 2;
- добавлен полносвязный слой после рекуррентного;  
- подкорректирован размер аутпута у слоя Embedding;  
- снижено количество юнитов LSTM с 128 до 16;  
- убран dropout у LSTM, вместо этого добавлен dropout-слой  
после полносвязного слоя;  
- повышено количество нейронов выходного слоя с 1 до 2,  
и, соответственно, изменена функция потерь с бинарной на  
категориальную.

Однако данный результат не превзошёл полученную в практическом  
задании к уроку № 3 метрику на том же самом датасете. В уроке № 3  
мы применяли полносвязные слои без рекуррентных. Скорее всего, это  
связано с тем, что в отзывах, которые являются признаками в этом  
датасете, порядок слов не имеет особой важности. Также следует отметить,  
что процесс обучения той нейросети шёл в 25 раз быстрее.

**Задание 2.**

*Измените параметры нейронной сети, генерирующей текст, таким образом, чтобы  
добиться генерации как можно более осмысленного текста. Опишите, что вы предприняли,  
чтобы его получить. Можно использовать текст другого прозведения.*

За основу возьмём модель с урока:

In [4]:
with open("../../Теория/Lesson_5/alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)

char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=bool)
y = np.zeros((len(input_chars), nb_chars), dtype=bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 64, 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 3
NUM_PREDS_PER_EPOCH = 100


model = Sequential()
model.add(
    GRU(HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam")


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print(f"Итерация #: {iteration+1}")
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    
    if iteration % 5 == 0 or iteration == NUM_ITERATIONS - 1:
        print("Генерация из посева: %s" % (test_chars))
        print(test_chars, end="")
        for i in range(NUM_PREDS_PER_EPOCH):

            # здесь one-hot encoding.
            X_test = np.zeros((1, SEQLEN, nb_chars))
            for j, ch in enumerate(test_chars):
                X_test[0, j, char2index[ch]] = 1

            # осуществление предсказания с помощью текущей модели.
            pred = model.predict(X_test, verbose=0)[0]
            y_pred = index2char[np.argmax(pred)]

            # вывод предсказания добавленного к тестовому примеру 
            print(y_pred, end="")

            # инкрементация тестового примера содержащего предсказание
            test_chars = test_chars[1:] + y_pred
print()

Итерация #: 1
Epoch 1/3
2481/2481 [==============================] - 11s 4ms/step - loss: 2.2021
Epoch 2/3
2481/2481 [==============================] - 9s 4ms/step - loss: 1.7804
Epoch 3/3
2481/2481 [==============================] - 9s 4ms/step - loss: 1.6090
Генерация из посева: ale, and s
ale, and she was a little said the could not in the could not in the could not in the could not in the could n==================================================
Итерация #: 2
Epoch 1/3
2481/2481 [==============================] - 9s 4ms/step - loss: 1.5003
Epoch 2/3
2481/2481 [==============================] - 9s 4ms/step - loss: 1.4246
Epoch 3/3
2481/2481 [==============================] - 9s 4ms/step - loss: 1.3681
Итерация #: 3
Epoch 1/3
2481/2481 [==============================] - 9s 4ms/step - loss: 1.3227
Epoch 2/3
2481/2481 [==============================] - 9s 4ms/step - loss: 1.2852
Epoch 3/3
2481/2481 [==============================] - 9s 4ms/step - loss: 1.2537
Итерация #: 4
Epoch 1/3
24

<ins>Вывод:</ins>  
Текст получился более-менее осмысленным. Поменял метод  
оптимизации на Adam, а также увеличил количество эпох до 3  
в каждой итерации и количество итераций до 25.  
Пробовал вместо GRU использовать LSTM и SimpleRNN, но  
величина ошибки на них была выше.